#### 전처리된 데이터 파일 로딩

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
X_train = pd.read_csv("./data/salary/preprocessing_X_train.csv",
                     index_col='no')
X_train.head()

,age,age_cut,capital-gain,capital-loss,education-num,hours-per-week,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Female,sex_ Male
no,,,,,,,,,,,,,,
1,25,2,0,0,7,40,0,1,0,0,0,0,0,1
2,39,4,0,1721,10,55,0,1,0,0,0,0,1,0
3,35,3,0,0,13,50,0,1,0,0,0,0,1,0
4,64,7,0,0,11,40,0,1,0,0,0,0,1,0
5,24,2,0,0,10,70,1,0,0,0,0,0,0,1


In [3]:
X_test = pd.read_csv("./data/salary/preprocessing_X_test.csv",
                     index_col='no')
X_test.head()

,age,age_cut,capital-gain,capital-loss,education-num,hours-per-week,relationship_ Husband,relationship_ Not-in-family,relationship_ Other-relative,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,sex_ Female,sex_ Male
no,,,,,,,,,,,,,,
29306,18,1,0,0,10,16,0,0,0,1,0,0,0,1
29307,29,3,0,0,9,40,0,0,0,0,0,1,1,0
29308,45,5,0,0,13,40,1,0,0,0,0,0,0,1
29309,42,5,0,0,13,40,1,0,0,0,0,0,0,1
29310,41,5,0,1902,16,40,0,0,0,0,0,1,1,0


In [4]:
y_train = pd.read_csv("./data/salary/y_train.csv",
                     index_col='no')
y_train.head()

,income
no,
1,0
2,0
3,0
4,0
5,1


### 목표
- 앙상블 모델의 개념을 이해
- 교차검증에 대한 이해
- GridSearch(최적화를 도와줌)를 이용한 하이퍼파라미터 튜닝 실습

#### 앙상블
- 단일 모델로 예측한 결과보다 더 좋은 성능을 내기위해 여러 모델을 묶어서 예측해보는 기법
- 앙상블 기법
    1. 보팅(Voting)
        - 서로다른 알고리즘의 모델을 학습시켜 예측결과를 투표하는 방법
        - 앙상블의 가장 기본 원리
        - 하드보팅(결과로 투포)과 소프트보팅(평균으로 결정)으로 나뉜다.
    2. 배깅(Bagging)
        - 같은 알고리즘을 가진 모델을 학습시켜 예측결과를 투표하는 방법
        - 학습데이터를 모델마다 다르게 준비(부트스트래핑)
        - 학습에 사용되는 컬럼을 다르게 랜덤선택
        - tree기반의 배깅모델 -> 랜덤포레스트
        - 병렬학습을 해서 상대적으로 속도가 빠르다.
    3. 부스팅(Boosting)
        - 같은 알고리즘을 가진 모델을 학습시켜 예측결과를 투표하는 방법
        - 이전 모델이 잘못 예측한 부분을 다음 모델이 강조해서 학습하는 방법사용
        - 순차적인 학습을 하기때문에 속도가 느리다.

#### 배깅모델

In [5]:
from sklearn.ensemble import RandomForestClassifier

In [6]:
rf_model = RandomForestClassifier(random_state=923, # 랜덤 수 고정
                                 n_estimators=1000, # 앙상블할 모델의 갯수 
                                 max_features=0.6, # 학습에 사용할 특성 수
                                 max_depth=15) # 트리의 깊이 설정 

In [7]:
rf_model.fit(X_train,y_train)

C:\Users\pipi1\AppData\Local\Temp\ipykernel_1788\764246711.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train,y_train)


RandomForestClassifier(max_depth=15, max_features=0.6, n_estimators=1000,
                       random_state=923)

In [8]:
rf_pre = rf_model.predict(X_test)

In [9]:
# 결과 업로드
submission = pd.read_csv("./data/salary/sample_submission.csv")
submission["income"] = rf_pre
submission.to_csv("./data/salary/rf_prediction1.csv",
                 index=False)

FileNotFoundError: [Errno 2] No such file or directory: './data/salary/gender_submission.csv'

#### 교차검증
- 모델의 일반화를 위해서 데이터를 훈련, 검증, 평가 셋으로 분리
- 검증셋이 여러개 있어서 모델의 신뢰도를 확보 할 수 있으면 좋다.
- 하지만 데이터양이 충분하지 않은 경우가 많아서 교차검증 기법을 많이 사용
- 교차검증 기법중에서도 k-fold cross validation기법이 일반적으로 쓰임

In [ ]:
from sklearn.model_selection import cross_val_score
cross_score = cross_val_score(rf_model,X_train,y_train,cv=5)

C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:680: DataConversionWarning: A column-vector y wa

In [ ]:
cross_score.mean()

0.8192894356914193

#### GridSearch를 이용한 모델최적화(하이퍼파라미터튜닝)
- 특정 검증데이터에서만 좋은 하이퍼파라미터보다는 
- 여러 검증데이터에서 좋은 하이퍼파라미터를 선택하는게 일반화에 도움이 되기 떄문에 모델최적화에서도 교차검증을 적용해서 사용한다.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# 모델 최적화(하이퍼파라미터튜닝)를 할 객체 생성
model2 = RandomForestClassifier(random_state=0)
# 파라미터 조합
pram_grid = {
    'max_depth' : [5, 10, 15, 20],
    'n_estimators' : [500, 1000, 1500, 2000],
    'max_features' : [0.5, 0.7, 0.8]
}  # 총 4x4x3 개

In [ ]:
grid = GridSearchCV(model2,   #튜닝할 모델
                    pram_grid, # 튜닝해볼 파라미터 조합
                    cv=3, # 교차검증 횟수
                    n_jobs=-1)  # 연산에 사용할 pc코어 수(-1 = 모든 피씨의 연산을 여기에 집중시키겠다)
                    # 총 파라미터 조합갯수 x 교차검증횟수(cv)

In [ ]:
grid.fit(X_train,y_train)

C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=926),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, 20],
                         'max_features': [0.5, 0.7, 0.8],
                         'n_estimators': [500, 1000, 1500, 2000]})

In [ ]:
print("최고 점수",grid.best_score_)
print("최고 조합",grid.best_params_)

최고 점수 0.8395061728395062
최고 조합 {'max_depth': 5, 'max_features': 0.8, 'n_estimators': 2000}


In [ ]:
# 학습이 완료된 최고 조합 모델
best_rf = grid.best_estimator_

In [ ]:
pre_test=best_rf.predict(X_test)

In [ ]:
# 결과 업로드
submission = pd.read_csv("./data/salary/sample_submission.csv")
submission["income"] = pre_test
submission.to_csv("./data/salary/rf_prediction2.csv",
                 index=False)

#### 부스팅 모델

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gbtree = GradientBoostingClassifier(n_estimators=100, # 나무의 갯수, 순차적으로 나무가 100번 만들어짐
                                    learning_rate=0.1, # 다음 트리에 반영할 오차 정도, 이전 트리에서 잘못한것을 다음트리한테 얼마나 반영할건지(0.1= 10%만 반영)
                                    max_depth=5, #나무의 최대깊이
                                    subsample=1 #학슴에 사용할 데이터의 비율
                                   )

In [ ]:
gbtree.fit(X_train,y_train)

C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(max_depth=5, subsample=1)

In [ ]:
pre_gb = gbtree.predict(X_test)
# 결과 업로드
submission = pd.read_csv("./data/salary/sample_submission.csv")
submission["income"] = pre_gb
submission.to_csv("./data/salary/gb_prediction2.csv",
                 index=False)


In [ ]:
# 그리드서치 이용한 모델 최적화 실습
# 모델 최적화(하이퍼파라미터튜닝)를 할 객체 생성
gb_model = GradientBoostingClassifier(random_state=0)
# 파라미터 조합
pram_grid = {
    'learning_rate' : [0.01, 0.1, 0.5, 0.8],
    'n_estimators' : [200, 500, 1000, 1500, 2000],
    'max_depth' : [5,7,9],
    'max_features' : [0.5, 0.7, 0.8],
    'subsample' : [0.5,0.7,1.0]
}  # 총 4x5x3x3x3x(cv)3 의 조합

In [ ]:
grid = GridSearchCV(gb_model,   #튜닝할 모델
                    pram_grid, # 튜닝해볼 파라미터 조합
                    cv=3, # 교차검증 횟수
                    n_jobs=-1)  # 연산에 사용할 pc코어 수(-1 = 모든 피씨의 연산을 여기에 집중시키겠다)

In [ ]:
grid.fit(X_train,y_train)

C:\Users\pipi1\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=3, estimator=GradientBoostingClassifier(random_state=0),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.5, 0.8],
                         'max_depth': [5, 7, 9],
                         'max_features': [0.5, 0.7, 0.8],
                         'n_estimators': [200, 500, 1000, 1500, 2000],
                         'subsample': [0.5, 0.7, 1.0]})

In [ ]:
pre_grid = grid.predict(X_test)


In [ ]:
print("최고 점수",grid.best_score_)
print("최고 조합",grid.best_params_)

최고 점수 0.8361391694725029
최고 조합 {'learning_rate': 0.01, 'max_depth': 5, 'max_features': 0.5, 'n_estimators': 500, 'subsample': 0.5}


In [ ]:
# 학습이 완료된 최고 조합 모델
best_rf = grid.best_estimator_
pre_test=best_rf.predict(X_test)

In [ ]:
# 결과 업로드
submission["income"] = pre_grid
submission.to_csv("./data/salary/gb_prediction2.csv",
                 index=False)

#### 외부 부스팅 라이브러리 사용하기
- boosting의 단점을 보완하고 추가 기능들을 탑재한 외부부스팅 라이브러리들이 있다
- ex) wboost, lightGBM, catboost등

In [ ]:
!pip install xgboost  # gpu에서 병렬 연산을 할수 있게 해주는 # 내부는 C나 c++로 되어있는데 겉은 파이썬
!pip install lightgbm  

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier(booster="gbtree", #부스팅할 모델 알고리즘 (선형모델 기반으로 하려면 gblinear)
                          learning_rate=0.1, # 학습률
                          max_depth=6, # 나무의 최대깊이
                          gamma=0.2) # 리프노드 분리를 결정하는 파라미터  
                                 # 키우면 모델 단순, 작아지면 모델 복잡

In [ ]:
xgb_model.fit(X_train,y_train)
pre_xgb = xgb_model.predict(X_test)